In [1]:
import tensorflow as tf
import os
import numpy as np

In [2]:
base_dir=r"/content/drive/MyDrive/capstone/all_img"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#preprocessing

In [5]:
IMAGE_SIZE=224
BATCH_SIZE=64

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(

           rescale=1./255,
           zoom_range=0.2,
           horizontal_flip=True,
           validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
         rescale=1./255,
         validation_split=0.1,
)

In [6]:
train_genarator=train_datagen.flow_from_directory(
       base_dir,
       target_size=(IMAGE_SIZE,IMAGE_SIZE),
       batch_size=BATCH_SIZE,
       subset='training'

    )

validation_generator=validation_datagen.flow_from_directory(
       base_dir,
       target_size=(IMAGE_SIZE,IMAGE_SIZE),
       batch_size=BATCH_SIZE,
       subset='validation'

    )

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [7]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob


In [8]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

58889256/58889256 [==============================] - 4s 0us/step


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
for layer in vgg.layers:
  layer.trainable=False

In [10]:
folders=glob(r"/content/vehicle/*")
print(len(folders))

0


In [11]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
epoch=10

history=model.fit(train_genarator,
                  steps_per_epoch=len(train_genarator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                     )

ValueError: ignored

In [ ]:
#prediction

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_pred = image.load_img(r"/content/Ambulance.yolo/63H17H33KXHS.jpg", target_size=(224, 224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0)

# Make predictions
result = model.predict(img_pred)

# Print the prediction
if result[0][0] > result[0][1]:
     prediction = "ambulance"
else:
     prediction = "vehicle"

print(prediction)
